# Create training sub set
Create a random subset of training and val data either on a balanced set of chexpert diseases or on just one disease.

In [26]:
#hermione created: 14/07/2023
import pandas as pd
import numpy as np


In [27]:

train = pd.read_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/train.csv")
chexpert = pd.read_csv("/home/hermione/Documents/data/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-chexpert.csv")
val = pd.read_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/valid.csv")
test = pd.read_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/test.csv")
train_chex = pd.merge(train, chexpert, on=['subject_id', 'study_id'])
val_chex = pd.merge(val, chexpert, on=['subject_id', 'study_id'])
test_chex = pd.merge(test, chexpert, on=['subject_id', 'study_id'])

#summary = pd.DataFrame(columns = chexpert.columns[2:], index = ["chexpert", "train", "val"])
summary = pd.DataFrame(columns = chexpert.columns[2:], index = ["chexpert", "train", "val","test"])
diseases = chexpert.columns[2:]

for col in diseases:
    #print(col)
    def dis_sum(data):
        return np.sum(data[data[col]==1][col].astype(int))
    """ def dis_frac(data):
        subset = data.iloc[2:]
        denom = int(sum([(subset[subset[d] == 1][d].astype(int)).sum() for d in diseases]))
        frac = dis_sum(data)/denom
        return frac """
    #summary[col] = [dis_sum(chexpert), dis_sum(train_chex), dis_sum(val_chex)]
    summary[col] = [dis_sum(chexpert), dis_sum(train_chex), dis_sum(val_chex), dis_sum(test_chex)]
    #print(np.sum(chexpert[chexpert[col]==1][col].astype(int)))
    #print(np.sum(train_chex[train_chex[col]==1][col].astype(int)))
    #print(np.sum(val_chex[val_chex[col]==1][col].astype(int)))

print(len(chexpert), len(train), len(val), len(train))
print(len(chexpert), len(train_chex), len(val_chex), len(train_chex))
#print(summary.sum(axis=1))
summary["total"] = summary.sum(axis=1)

summary.T #potentially make percentage
#summary.to_csv("summary.csv", index=False)

227827 33510 4894 33510
227827 33510 4894 33510


,chexpert,train,val,test
Atelectasis,45808,5110,753,1580
Cardiomegaly,44845,4036,611,1340
Consolidation,10778,856,104,276
Edema,27018,3003,499,993
Enlarged Cardiomediastinum,7179,613,70,191
Fracture,4390,638,75,170
Lung Lesion,6284,801,111,236
Lung Opacity,51525,6631,984,2056
No Finding,75455,14514,2055,4305
Pleural Effusion,54300,5595,825,1701


In [25]:
print(train_chex[train_chex["Pleural Effusion"] == 1].iloc[8,3])
print(train_chex[train_chex["Pleural Effusion"] == 1].iloc[8,4])
train_chex[train_chex["Pleural Effusion"] == 1].iloc[:,:5]

/home/hermione/Documents/data/physionet.org/files/mimic-cxr-jpg/2.0.0/files/p10/p10002428/s54831516/852306b6-02fc04aa-82d30dbf-0c2dd18d-5c9ef054.jpg
Comparison is made to previous study from ___. There is a Dobbhoff tube whose distal tip is in the body of the stomach. There are bilateral pleural effusions. There is a right-sided pleural-based catheter. There is no pneumothoraces or signs for overt pulmonary edema. Overall, these findings are stable since prior study from ___.


,subject_id,study_id,image_id,mimic_image_file_path,reference_report
4,10000980,57861150,5aa15ba6-55f5e96e-39cea686-7c3b28b2-b8c97a88,/home/hermione/Documents/data/physionet.org/fi...,Heart size remains mild to moderately enlarged...
5,10000980,58206436,54affd39-8bf24209-232bac8a-df6c277a-398ee8a5,/home/hermione/Documents/data/physionet.org/fi...,In comparison to study performed on of ___ the...
6,10000980,59988438,925b9496-a956d7b2-05185e52-bb33313b-c06ee522,/home/hermione/Documents/data/physionet.org/fi...,PA and lateral views of the chest demonstrate ...
18,10002013,58054149,81bca127-0c416084-67f8033c-ecb26476-6d1ecf60,/home/hermione/Documents/data/physionet.org/fi...,A moderate left pleural effusion is new since ...
19,10002131,52823782,051b7911-cb00aec9-0b309188-89803662-303ec278,/home/hermione/Documents/data/physionet.org/fi...,Frontal and lateral chest radiographs demonstr...
...,...,...,...,...,...
33407,12005748,54314058,31934fa6-8a51ab44-b81b1eac-522b7757-478c4ff4,/home/hermione/Documents/data/physionet.org/fi...,There is cardiomegaly accompanied by vascular ...
33455,12461334,59813777,9e882192-2860e30b-bc1729f0-f3dd9119-f4ef962a,/home/hermione/Documents/data/physionet.org/fi...,There is leftward effusion increased opacity i...
33463,12509458,51239211,964cc30a-f4549f80-99ecfb9c-d498554e-2db3da2d,/home/hermione/Documents/data/physionet.org/fi...,There is moderate cardiomegaly. The patient is...
33488,12738545,54211667,b80f54be-2c4c55f5-703ebbe3-d610f02f-9613a483,/home/hermione/Documents/data/physionet.org/fi...,Moderate cardiomegaly persists. Aortic knob is...


In [6]:

#summary_frac = pd.DataFrame(columns=summary.columns[:-1], index=summary.index)
summary_frac = []
for row in summary.index:
    columns =[]
    for col in diseases:
        def frac(data):
            return np.round(data.loc[row,col]/data.loc[row, "total"],3)
        columns.append(frac(summary))
    summary_frac.append(columns)

summary_frac_df = pd.DataFrame(summary_frac ,columns=summary.columns[:-1], index=summary.index)
summary_frac_df.T.to_csv("summary_frac.csv", index=False)
summary_frac_df.T

,chexpert,train,val,test
Atelectasis,0.108,0.099,0.100,0.100
Cardiomegaly,0.106,0.078,0.081,0.085
Consolidation,0.025,0.017,0.014,0.018
Edema,0.064,0.058,0.067,0.063
Enlarged Cardiomediastinum,0.017,0.012,0.009,0.012
Fracture,0.010,0.012,0.010,0.011
Lung Lesion,0.015,0.016,0.015,0.015
Lung Opacity,0.122,0.129,0.131,0.131
No Finding,0.178,0.282,0.274,0.273
Pleural Effusion,0.128,0.109,0.110,0.108


In [15]:
#select data
train_data_index = train_chex[train_chex["No Finding"] == 1].sample(frac=(1/3), random_state=1).index
val_data_index = val_chex[val_chex["No Finding"] == 1].sample(frac=(1/3), random_state=1).index
train_data = train_chex[train_chex.index.isin(train_data_index)| (train_chex["Pleural Effusion"] == 1)]

val_data = val_chex[val_chex.index.isin(val_data_index) | (val_chex["Pleural Effusion"] == 1)]

print(train_data.columns, len(train_data))
train_final = train_data[['subject_id', 'study_id', 'image_id', 'mimic_image_file_path','reference_report']]
val_final = val_data[['subject_id', 'study_id', 'image_id', 'mimic_image_file_path','reference_report']]
print(len(train_final), len(val_final))

Index(['subject_id', 'study_id', 'image_id', 'mimic_image_file_path',
       'reference_report', 'Atelectasis', 'Cardiomegaly', 'Consolidation',
       'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
       'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
       'Pneumonia', 'Pneumothorax', 'Support Devices'],
      dtype='object') 10433
10433 1510


In [4]:
train_final.to_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/train_subset.csv",index=False)
val_final.to_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/valid_subset.csv", index=False)

In [28]:
#select data
train_data= train_chex[(train_chex["Cardiomegaly"] == 1) | (train_chex["Pleural Effusion"] == 1)] #.sample(n=10000, random_state=1).index, (train_chex["Cardiomegaly"] == 1) | 
val_data= val_chex[(val_chex["Cardiomegaly"] == 1) |(val_chex["Pleural Effusion"] == 1)] #.sample(n=1500, random_state=1).index, (val_chex["Cardiomegaly"] == 1) | 
test_data= test_chex[(test_chex["Cardiomegaly"] == 1) |(test_chex["Pleural Effusion"] == 1)] 
#train_data = train_chex[~train_chex.index.isin(train_data_index)]
#val_data = val_chex[~val_chex.index.isin(val_data_index)]

print(train_data.columns, len(train_data))
train_final = train_data[['subject_id', 'study_id', 'image_id', 'mimic_image_file_path','reference_report']]
val_final = val_data[['subject_id', 'study_id', 'image_id', 'mimic_image_file_path','reference_report']]
test_final = test_data[['subject_id', 'study_id', 'image_id', 'mimic_image_file_path','reference_report']]
print(len(train_final), len(val_final))

#train_final.to_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/train_PE.csv",index=False)
#val_final.to_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/valid_PE.csv", index=False)
test_final.to_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports/test_card_vs_PE.csv", index=False)

Index(['subject_id', 'study_id', 'image_id', 'mimic_image_file_path',
       'reference_report', 'Atelectasis', 'Cardiomegaly', 'Consolidation',
       'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
       'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
       'Pneumonia', 'Pneumothorax', 'Support Devices'],
      dtype='object') 8331
8331 1258


In [19]:
#definitly not overlapping
print(train_final[train_final["mimic_image_file_path"].isin(val_final["mimic_image_file_path"])])

Empty DataFrame
Columns: [subject_id, study_id, image_id, mimic_image_file_path, reference_report]
Index: []


# Remake Datasets from bounding box reports
This shouls hopefully only keep pathology relevant line rather than scan veiws or many references to previos scans. 
Well i tried

In [49]:
from ast import literal_eval
train_bb = pd.read_csv("/home/hermione/Documents/data/rgrg/dataset-with-reference-reports_original/train.csv").drop(columns=["bbox_coordinates","bbox_labels","bbox_is_abnormal", "bbox_phrase_exists"])
train_bb["reference_report"] = [". ".join(list(set([i.lstrip() for i in (" ".join(" ".join(literal_eval(phrases)).split()).split("."))]))) for phrases in train_bb["bbox_phrases"]]


In [46]:

train_bb = train_bb.drop(columns=["bbox_phrases"])
print(len(train_bb))
val_bb = train_bb.sample(frac=0.1, random_state=1)
train_bb = train_bb[~train_bb["image_id"].isin(val_bb["image_id"])]
test_bb = train_bb.sample(n=len(val_bb), random_state=1)
train_bb = train_bb[~train_bb["image_id"].isin(test_bb["image_id"])]
val_bb

49311


,subject_id,study_id,image_id,mimic_image_file_path,reference_report
18647,11172056,50366239,324f2928-3d040888-70b3742c-f2c3b988-63cc7617,/home/hermione/Documents/data/physionet.org/fi...,. No pneumothorax or effusion. New right upper...
48959,10766641,55573119,abbfc4d2-4c08ecd5-95216c83-f9e7e8d4-f0e03a44,/home/hermione/Documents/data/physionet.org/fi...,. The heart size and cardiomediastinal contour...
22567,11400517,56972034,f60572ea-5c0e131d-0be53637-2333841e-2c930ea3,/home/hermione/Documents/data/physionet.org/fi...,. The cardiac and mediastinal silhouettes are ...
35385,12144619,58964820,bf5c0d9f-b4cb1207-607aacb3-f6a24029-04dec268,/home/hermione/Documents/data/physionet.org/fi...,. Borderline size of the cardiac silhouette. N...
34886,12124186,52107348,dbf88c6a-071650e5-0ec82dc7-b4f58f7f-54c13ee5,/home/hermione/Documents/data/physionet.org/fi...,"No focal consolidation, pleural effusion or pn..."
...,...,...,...,...,...
21826,11346699,50584318,617afa71-673719dc-0dbecb50-028de5c8-38e3434e,/home/hermione/Documents/data/physionet.org/fi...,. The degree of pulmonary edema appears to be ...
14664,10922531,51051513,8646ffe3-5b8b6570-bbc7fb58-c8489fe0-9af4b9ac,/home/hermione/Documents/data/physionet.org/fi...,. The heart is top- normal size given the larg...
9265,10584015,51857744,ceca7221-1f04d4e0-e3048c54-6671d2f4-5faf37e6,/home/hermione/Documents/data/physionet.org/fi...,. The hilar and mediastinal structures are unr...
5520,10353355,53929101,397b0512-87c35ce7-490c463c-51fc61fb-07eb75e6,/home/hermione/Documents/data/physionet.org/fi...,. Left lower lobe pneumonia. There is interval...


In [37]:
print(len(train_bb), len(val_bb), len(test_bb))
print(len(train), len(val), len(test))

39449 4931 4931
33510 4894 10147


In [50]:
print(train_bb["reference_report"][0])
print(train_bb["reference_report"][1])
#print(train_bb["bbox_phrases"][0])
train_bb

. Heart is top-normal size, improved, and pleural effusions have resolved. Previous mild pulmonary edema and possible concurrent pneumonia has all cleared. Lateral view shows atherosclerotic coronary calcification in the left circumflex. Right hilar vessels are still enlarged, perhaps due to pulmonary arterial hypertension
There is also fullness of the right hilum which is new. . The heart size is top normal. However, given right hilar fullness, a mass resulting in post-obstructive pneumonia is within the differential. Right upper lobe pneumonia or mass. Blunting of bilateral costophrenic angles, right greater than left, may be secondary to small effusions. A triangular opacity in the right lung apex is new from prior examination. The remainder of the lungs are clear


,subject_id,study_id,image_id,mimic_image_file_path,bbox_phrases,reference_report
0,10000980,50985099,6ad03ed1-97ee17ee-9cf8b320-f7011003-cd93b42d,/home/hermione/Documents/data/physionet.org/fi...,['Previous mild pulmonary edema and possible c...,". Heart is top-normal size, improved, and pleu..."
1,10000980,51967283,943486a3-b3fa9ff7-50f5a769-7a62fcbb-f39b6da4,/home/hermione/Documents/data/physionet.org/fi...,['A triangular opacity in the right lung apex ...,There is also fullness of the right hilum whic...
2,10000980,54577367,cfb03587-782edf6c-1bf392e1-98196cd5-365d69e8,/home/hermione/Documents/data/physionet.org/fi...,['There is minimal upper zone vascular redistr...,. There is minimal upper zone vascular redistr...
3,10000980,54935705,6ad819bb-bae74eb9-7b663e90-b8deabd7-57f8054a,/home/hermione/Documents/data/physionet.org/fi...,['There are possible small bilateral pleural e...,. There is mild pulmonary edema with superimpo...
4,10000980,54980801,a75a1fbe-802065ad-717eb7c1-e2ce3552-646276a6,/home/hermione/Documents/data/physionet.org/fi...,"['The lungs are clear of consolidation, effusi...",. No acute cardiopulmonary process. Cardiac si...
...,...,...,...,...,...,...
49306,12958472,52582935,e689d7cb-52bc8526-f000f9b6-1ec8ebe2-0ea8e4f6,/home/hermione/Documents/data/physionet.org/fi...,"['Pulmonary edema, moderate right pleural effu...",". The heart is top normal size, exaggerated by..."
49307,12958472,57693298,ef33ed50-3103c924-869900f8-dfb73dd3-8b1f0c0a,/home/hermione/Documents/data/physionet.org/fi...,['Moderate right pleural effusion and multifoc...,. There is also at least mild pulmonary edema....
49308,12963147,56053808,665650b7-273d4330-d61f391c-8a4663ab-22ae3715,/home/hermione/Documents/data/physionet.org/fi...,['Lungs are low in volume resulting in broncho...,. Mild vascular congestion. Bibasilar linear a...
49309,12989742,59169029,d072a686-274dbd4a-aa1d2b90-61ba527b-d4b9ebdd,/home/hermione/Documents/data/physionet.org/fi...,"['The lungs appear lucent in the upper lobes, ...",. Subtle reticular opacities in the left lower...
